In [ ]:
import os

# ----- Option 1: Using OpenAI GPT-4 (Chat Model) -----

# OPENAI_API_KEY = "API KEY"  # Place your API key here

# from langchain_community.chat_models import ChatOpenAI
# MODEL = "gpt-4"
# model = ChatOpenAI(model_name=MODEL, openai_api_key=OPENAI_API_KEY)

# from langchain.embeddings import OpenAIEmbeddings
# embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)


# ----- Option 2: Using Local Ollama Model (llama3.1:8b) -----
# Comment out this block if you want to use OpenAI instead.
from langchain_community.llms import Ollama
from langchain_ollama import OllamaEmbeddings

#MODEL = "llama3.1:8b"  # Change this if needed
MODEL = "deepseek-r1:8b"  # Change this if needed
model = Ollama(model=MODEL)
embeddings = OllamaEmbeddings(model=MODEL)


C:\Users\mizan\AppData\Local\Temp\ipykernel_16936\61173007.py:22: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  model = Ollama(model=MODEL)


In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import DocArrayInMemorySearch

# Load and split the PDF into pages
loader = PyPDFLoader("ragtest.pdf")
pages = loader.load_and_split()

# Create an in-memory vector store from the PDF pages using the embeddings
vectorstore = DocArrayInMemorySearch.from_documents(pages, embedding=embeddings)
retriever = vectorstore.as_retriever()

In [9]:
# Cell 3: Build the Composite Chain
# =====================
from langchain.prompts import PromptTemplate
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser

# Define the prompt template
template = """
Answer the question based on the context below. If you can't 
answer the question, reply "I don't know".

Context: {context}

Question: {question}
"""
prompt = PromptTemplate.from_template(template)
parser = StrOutputParser()

# Create a composite chain:
# 1. Retrieve context based on the question.
# 2. Format the prompt.
# 3. Get an answer from the model and parse it.
chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
    }
    | prompt
    | model
    | parser
)

In [10]:
# Cell 4: Ask a Question & Stream the Answer
# =====================
# Change the question here as needed.
question_input = {"question": "What does the Chronos Amulet do?"}

# Stream the answer and print it in real time
for s in chain.stream(question_input):
    print(s, end="", flush=True)
print()  # Ensure a newline at the end

The Chronos Amulet grants its wearer the ability to manipulate time.
